<a href="https://colab.research.google.com/github/futureCodersSE/Coding-workshop-resources/blob/master/Music-Animation/animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Animation created from sampled music (30 second clip) using matplotlib and a scatter diagram
---


## To create the animation:

* Run cells 0, 1, 2 and 3  
* Check after cell 3 that you have a sound_data sample that has a length and a minimum of 0 and maximum of 100 (individual sounds will be represented by a number between 0 and 100% of the maximum

# Section A - setting up and getting sound data
----
**Upload the sound data**

* install and import the libraries (_these do the work so you don't have to code this part_)
* open the sound data file (_you can create a plain animation from a 30 second mp3 sound clip, or with a background image from a 15 second mp3 sound clip_)

**Once you have done this you will have a set of 'sound data' that you can create a chart from, then can animate it.**

* Run cells 0, 1,  and 2  

## 0. Install dependencies
---


In [ ]:
!pip install audio2numpy
!pip install IPython

## 1. Import libraries
---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc

import audio2numpy as a2n
from google.colab import files

import IPython

## 2. Read and store the sound data
---

* Upload the sound file from your computer.
* Turn the sound into data

Once you have done this, the sound data will be available until next time you run the worksheet.

In [ ]:
def get_sound_data():
  uploaded = files.upload()
  for fn in uploaded.keys():
    filename = fn
  x,sr=a2n.audio_from_file(filename)
  return x, filename


!rm *
data, SOUND_FILE = get_sound_data()

# Section B - process the sound data for the animation
---
* how many frames per second?
* how long is the sound clip?
* prepare the data to fit the frames per second

**Once you have done this you will have a set of data that is ready to animate**  

* Run cells 3 and 4 to prepare the data

* Check after cell 4 that you have a sound_data sample that has a length and a minimum of 0 and maximum of 100 (individual sounds will be represented by a number between 0 and 100% of the maximum)

##3. Define frames per second, clip length

In [ ]:
FRAMES_PER_SECOND = 10
CLIP_LENGTH = 28

##4. Process the sound data
---


In [ ]:
# Get sound samples, adjust up so no negatives, convert each to a percentage of the range
def process_data(data, clip_length, frames_per_second):
  sound_data = data[:,0]
  number_of_samples = clip_length  * frames_per_second
  sample_rate = len(sound_data)//number_of_samples
  # sample at intervals rather than every beat
  sound_data = sound_data[::sample_rate]
  if sound_data.min() < 0:
    sound_data = sound_data + abs(sound_data.min())
  sound_data = (sound_data - sound_data.min()) / (sound_data.max() - sound_data.min()) * 100
  return sound_data

# set number of beats in sound clip at sampling rate set in EarSketch (e.g. tempo)
SOUND_DATA = process_data(data, CLIP_LENGTH, FRAMES_PER_SECOND)
print("Number of sound samples =", len(SOUND_DATA))
print("Minimum value =", SOUND_DATA.min())
print("Maximum value =", SOUND_DATA.max())

# Section C - customise the animation settings
---

Set the background colour, colour, shape and size of moving objects, number of moving objects, any background image

**This animation will be a graph representing the data, that will change as the data changes.**

Run cells 5 and 6 to customise the animation

## 5. Set animation features (colours, sizes, number of objects, number of frames, interval between frames)

[**Information about available colours**](https://drive.google.com/file/d/1YAlx3IXOJ_KXreKAlnNCeINvOKcciXz4/view?usp=sharing)  

[**Information about marker styles**](https://matplotlib.org/stable/api/markers_api.html)  

Marker size is in pixels  

In [ ]:
# set colour theme
BACKGROUND_COLOUR = 'tab:olive'
MARKER_COLOURS = "darkslategray"  # if the markers will be different colours, make a list of colours with a colour for each marker (so a list of 10 colours for 10 markers)

# set graphics
NUMBER_OF_MARKERS = 10
MARKER_STYLE = "h"
MARKER_SIZE = 200

# set timing (number of frames and length of time between each, to make animation same length as sound clip)
FRAMES = len(SOUND_DATA) - NUMBER_OF_MARKERS #
INTERVAL = 1000 // FRAMES_PER_SECOND  # number of milliseconds between frames

## 6. Upload a background image (optional and reduces the size of the animation you can make)
---

You can add a background image but this will make the animation big and so you will need to make it shorter.  You can use background colour rather than image to help keep the animation smaller.

If you want to use a background image, upload it by running cell 6

In [ ]:
# get the background image

def get_background_image():
  uploaded = files.upload()
  for fn in uploaded.keys():
    filename = fn
  return filename

!rm *
# BG_IMAGE = get_background_image()

# Section D - create the animation
---
Create a 'chart' from the first sound sample.  This will then be modified to reflect each sound sample in turn.

**Run cells**:  
7 - to make the first frame (this time - the frame is ten objects in a line plotted from the first sound sample in the file  
8 - to define how the second, third, etc frames will be created  
9 - build the animation file

## 7. Create the first frame
---


In [ ]:
# create plot
fig, ax = plt.subplots()
# img = plt.imread(BG_IMAGE) ## uncomment if adding a background image
# ax.imshow(img, aspect='auto', extent=[0,NUMBER_OF_MARKERS+1, 0, 100]) ## uncomment if adding a background image

# Hide X and Y axes label and tick marks
ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])

# Set background colour and the axes size
ax.set_facecolor(BACKGROUND_COLOUR)
ax.set(xlim=[0, NUMBER_OF_MARKERS+1], ylim=[0, 100])

# create a set of points to plot from the first sound sample
x = [i for i in range(1,NUMBER_OF_MARKERS+1)]
y = y = SOUND_DATA[:NUMBER_OF_MARKERS]
scatter = ax.scatter(x, y, c=MARKER_COLOURS, s=MARKER_SIZE, vmin=0, vmax=100, marker=MARKER_STYLE)


## 8. Define how the animation will work
---


In [ ]:
def update(frame):
    # for each frame, update the data stored on each artist, keeping x values the same.
    if frame + 10 > len(SOUND_DATA):
      frame = frame - 10
    y = SOUND_DATA[frame:frame+10]
    data = np.stack([x, y]).T
    scatter.set_offsets(data)
    return (scatter,)

## 9. Build the animation
---

It will take a minute or two to build, then you will be able to run it as many times as you want.  When you are happy with it you can save it.  To make changes, change settings in section 4 (Create a base plot), then run 4 and 5 again, before running the animation again to test it.

In [ ]:
rc('animation', html='jshtml')
anim = FuncAnimation(fig=fig, func=update, frames=FRAMES, interval=INTERVAL)
anim

## 10. Play the music (test that the animation fits)
---


In [ ]:
IPython.display.Audio(SOUND_FILE)

## 11.  Save the animation as a gif
---

In [ ]:
filename = "test_animation.gif"
anim.save(filename=filename, writer="pillow")
files.download(filename)

# Section E - Create a video using MovieMaker online
---
### To make into a movie/video
* go to https://moviemakeronline.com/mm/movie-maker
* upload the saved gif
* upload the music file
* make the movie
* download the mp4
* play the movie
